In [ ]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader

#%load_ext line_profiler

In [ ]:
from model.eGauss_plus import eGAUSSp

In [ ]:
# Load the dataset
file_path = 'Datasets/creditcard.csv'
data = pd.read_csv(file_path)


In [ ]:
from utils.utils_dataset import balance_dataset
from utils.utils_dataset import prepare_dataset

device = torch.device("cpu")

num_clients = 4

data = balance_dataset(data)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
client_train, test_data, all_data = prepare_dataset(X, y, num_clients) 


In [ ]:

# Model parameters
local_model_params = {
    "feature_dim": 30,
    "num_classes": 2,
    "N_max": 1,
    "num_sigma": 5,
    "kappa_join": 3,
    "S_0": 1e-8,
    "c_max": 1,
    "device": torch.device("cpu")
}
federated_model_params = {
    "feature_dim": 30,
    "num_classes": 2,
    "N_max": 1,
    "num_sigma": 4,
    "kappa_join": 3,
    "S_0": 1e-8,
    "c_max": 2,
    "device": torch.device("cpu")
}

# Initialize a model for each client
local_models = [eGAUSSp(**local_model_params) for _ in range(num_clients)]
federated_model = eGAUSSp(**federated_model_params)


In [ ]:
from utils.utils_dataset import prepare_non_iid_dataset, plot_dataset_split, display_dataset_split

display_dataset_split(client_train, test_data)
plot_dataset_split(client_train, test_data)

In [ ]:
import utils.utils_train
from utils.utils_train import train_supervised, train_unsupervised
from utils.utils_plots import plot_first_feature

if False:

    # Assuming models and client_data are already defined
    for client_idx, client_model in enumerate(local_models):
        print(f"Training model for client {client_idx + 1}")
        train_supervised(client_model, client_train[client_idx]) 
        plot_first_feature(client_train[client_idx], model=client_model, num_sigma=2, N_max=1)  


In [ ]:
import utils.utils_train
from utils.utils_train import train_models_in_threads
from utils.utils_plots import plot_first_feature

# Assuming local_models and client_train are already defined
# local_models is a list of models, client_train is a list of corresponding training datasets
train_models_in_threads(local_models, client_train)


In [ ]:
from utils.utils_train import test_model   

# Testing each client's model on the global test dataset
for client_idx, client_model in enumerate(local_models):
    print(f"\nTesting model for client {client_idx + 1}")
    metrics = test_model(client_model, test_data)
    #test_model(client_model, all_data)
    print(f"Test Metrics: {metrics}")

In [ ]:

for client_idx, client_model in enumerate(local_models):
        print(f"Merging client {client_idx + 1}")
        client_model.federal_agent.federated_merging() 
        print(f"Number of client {client_idx + 1} clusters = {torch.sum(client_model.n[:client_model.c]>1)}")
        federated_model.federal_agent.merge_model_privately(client_model, 1)

print(f"Number of clusters after transfer = {federated_model.c}")

In [ ]:
from utils.utils_train import test_model

federated_model.federal_agent.federated_merging()
print(f"Number of clusters after merging = {federated_model.c}")

plot_first_feature(all_data, model=federated_model, num_sigma=2, N_max=1)   

print(f"\nTesting federated model")   

test_model(federated_model, test_data)
#test_model(federated_model, all_data)